<a href="https://colab.research.google.com/github/passer87/aidata/blob/main/Paxos%E5%8D%94%E8%AD%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Paxos協議
import threading

class Acceptor:
    def __init__(self):
        self.promised_n = None
        self.accepted_n = None
        self.accepted_value = None

    def prepare(self, n):
        if self.promised_n is None or n > self.promised_n:
            self.promised_n = n
            return True, self.accepted_n, self.accepted_value
        return False, self.accepted_n, self.accepted_value

    def accept(self, n, value):
        if self.promised_n is None or n >= self.promised_n:
            self.accepted_n = n
            self.accepted_value = value
            return True
        return False

class Proposer:
    def __init__(self, id, acceptors):
        self.id = id
        self.n = 0
        self.acceptors = acceptors

    def propose(self, value):
        self.n += 1
        promises = []
        for acceptor in self.acceptors:
            promise, accepted_n, accepted_value = acceptor.prepare(self.n)
            if promise:
                promises.append((accepted_n, accepted_value))

        if len(promises) > len(self.acceptors) // 2:
            accepted_value = value
            for accepted_n, val in promises:
                if val is not None:
                    accepted_value = val
            accept_count = 0
            for acceptor in self.acceptors:
                if acceptor.accept(self.n, accepted_value):
                    accept_count += 1
            if accept_count > len(self.acceptors) // 2:
                return accepted_value
        return None

class Learner:
    def __init__(self):
        self.decided_value = None

    def learn(self, value):
        self.decided_value = value
        print(f"學習者達成一致決策: {value}")
 # 測試Paxos協議
acceptors = [Acceptor() for _ in range(5)]
proposers = [Proposer(i, acceptors) for i in range(3)]
learner = Learner()

def proposer_thread(proposer, value):
    result = proposer.propose(value)
    if result is not None:
        learner.learn(result)

threads = []
for proposer in proposers:
    t = threading.Thread(target=proposer_thread, args=(proposer, f"Value from proposer {proposer.id}"))
    threads.append(t)
    t.start()

for t in threads:
    t.join()

學習者達成一致決策: Value from proposer 0
